In [39]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
from census import Census
from pprint import pprint
from Census_Field_Library import census_aliases
import gmaps

# Census API
from config import Census_API
c = Census(Census_API, year=2015)

In [26]:
pprint(census_aliases)

{'age': {'api': 'acs',
         'description': 'Median age',
         'text': 'median age',
         'unit': 'age',
         'variable': 'B01002_001E'},
 'commute_time_carpool': {'api': 'acs',
                          'description': 'Time spent commuting (in minutes): '
                                         'Car, truck, or van - carpool',
                          'normalizable': True,
                          'normalize_by': 'transit_carpool',
                          'text': 'commuting by carpool',
                          'unit': 'minutes',
                          'variable': 'B08136_004E'},
 'commute_time_other': {'api': 'acs',
                        'description': 'Time spent commuting (in minutes): '
                                       'Taxicab, motorcycle, bicycle, or other '
                                       'means',
                        'normalizable': True,
                        'normalize_by': 'transit_other',
                        'text': 'commuting

                                                  'description': 'Number of '
                                                                 'employed '
                                                                 "male 'Sales "
                                                                 'and office '
                                                                 "occupations:' "
                                                                 'for the '
                                                                 'civilian '
                                                                 'population '
                                                                 'age 16 and '
                                                                 'over',
                                                  'text': 'male office workers '
                                                          'and sales '
                                                          'associate

In [31]:
population_by_language_dict = {"English":"B16001_002E","Spanish":"B16001_003E",
                              "French":"B16001_006E","Italian":"B16001_012E",
                              "Portuguese":"B16001_015E","German":"B16001_018E",
                              "Greek":"B16001_030E","Russian":"B16001_033E","Polish":"B16001_036E",
                              "Chinese":"B16001_066E","Japanese":"B16001_069E","Korean":"B16001_072E",
                              "Thai":"B16001_081E","Vietnamese":"B16001_087E","Arabic":"B16001_108E"}

language_name_list = list(population_by_language_dict.keys())
language_aliaese_list = list(population_by_language_dict.values())

In [43]:
# Get City ZipCode Data
city_zipcode_df = pd.read_csv("zip_code_database.csv")
city_zipcode_df.head()

,zip,type,decommissioned,primary_city,acceptable_cities,unacceptable_cities,state,county,timezone,area_codes,world_region,country,latitude,longitude,irs_estimated_population_2015
0,501,UNIQUE,0,Holtsville,NaN,I R S Service Center,NY,Suffolk County,America/New_York,631,NaN,US,40.81,-73.04,562
1,544,UNIQUE,0,Holtsville,NaN,Irs Service Center,NY,Suffolk County,America/New_York,631,NaN,US,40.81,-73.04,0
2,601,STANDARD,0,Adjuntas,NaN,"Colinas Del Gigante, Jard De Adjuntas, Urb San...",PR,Adjuntas Municipio,America/Puerto_Rico,"787,939",NaN,US,18.16,-66.72,0
3,602,STANDARD,0,Aguada,NaN,"Alts De Aguada, Bo Guaniquilla, Comunidad Las ...",PR,Aguada Municipio,America/Puerto_Rico,"787,939",NaN,US,18.38,-67.18,0
4,603,STANDARD,0,Aguadilla,Ramey,"Bda Caban, Bda Esteves, Bo Borinquen, Bo Ceiba...",PR,Aguadilla Municipio,America/Puerto_Rico,787,NaN,US,18.43,-67.15,0


In [50]:
# Extract zip code data for Chicago Only
column_output = ["zip","primary_city","state","latitude","longitude"]
city_zipcode_filter_df = pd.DataFrame(columns=column_output)

city_zipcode_filter_df = city_zipcode_df.loc[(city_zipcode_df["primary_city"]=="Chicago")&(city_zipcode_df["state"]=="IL"),column_output]

zipcode_list = list(pd.to_numeric(city_zipcode_filter_df["zip"]))

In [53]:
# Run Census Search to retrieve data on all states
# Note the addition of "B23025_005E" for unemployment count
census_data = c.acs5.get(language_aliaese_list+["B01003_001E"], {'for': 'zip code tabulation area:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={**{"zip code tabulation area": "Zipcode","B01003_001E": "Population"},**dict(zip(language_aliaese_list,language_name_list))})


census_pd["Zipcode"] = pd.to_numeric(census_pd["Zipcode"])

census_Chicago_pd = census_pd.loc[census_pd["Zipcode"].isin(zipcode_list)]

census_Chicago_pd.head()

,Population,English,Spanish,French,Italian,Portuguese,German,Greek,Russian,Polish,Chinese,Japanese,Korean,Thai,Vietnamese,Arabic,Zipcode
20694,11905.0,7689.0,887.0,110.0,110.0,186.0,103.0,105.0,31.0,190.0,575.0,118.0,185.0,0.0,1.0,199.0,60601
20695,1294.0,1081.0,60.0,3.0,0.0,1.0,1.0,0.0,0.0,0.0,11.0,57.0,16.0,0.0,0.0,14.0,60602
20696,1076.0,687.0,90.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,43.0,4.0,143.0,0.0,19.0,14.0,60603
20697,545.0,388.0,42.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,5.0,60604
20698,27238.0,19660.0,973.0,422.0,283.0,155.0,71.0,32.0,232.0,122.0,1209.0,206.0,269.0,107.0,25.0,184.0,60605


In [54]:
census_Chicago_pd.to_csv("Population_by_Language_Chicago.csv",index=False)